In [ ]:
import torch
import torchvision
from torchvision import transforms
import torchvision.datasets as dset
from torch.utils.data import DataLoader,Dataset
import torch.optim as optim
from torch.autograd import Variable
import torch.nn as nn

import glob
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from prettytable import PrettyTable

In [ ]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
img_transforms=transforms.Compose([transforms.Resize((300,300)),
                              transforms.RandomHorizontalFlip(p=0.5),
                              transforms.ToTensor()
                             ])

In [ ]:
train_dir='./train_valid_test/train/'
valid_dir='./train_valid_test/valid/'
test_dir='./train_valid_test/test/'

train_loader=DataLoader(dset.ImageFolder(train_dir,transform=img_transforms),
                                        batch_size=64,shuffle=True)

valid_loader=DataLoader(dset.ImageFolder(valid_dir,transform=img_transforms),
                                        batch_size=64,shuffle=False)

test_loader=DataLoader(dset.ImageFolder(test_dir,transform=img_transforms),
                                        batch_size=1,shuffle=True)

In [ ]:
classes=['cardboard','glass','metal','paper','plastic','trash']

In [ ]:
train_count=len(glob.glob(train_dir+'/**/*.jpg'))
valid_count=len(glob.glob(valid_dir+'/**/*.jpg'))
test_count=len(glob.glob(test_dir+'/**/*.jpg'))
print("No of  images in\n\tTraining set : ",train_count,"\n\tValidation set : ",valid_count,"\n\tTest set : ",test_count)

In [ ]:
# neural network
class GarbageNet(nn.Module):
    def __init__(self):
        super(GarbageNet,self).__init__()
        #(64,3,300,300)
        self.conv=nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3,padding=1), #(64,32,300,300)
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2), #(64,32,150,150)
            nn.Dropout(),
        
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3), #(64,64,148,148)
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2), #(64,64,74,74)
        
            nn.Conv2d(in_channels=64, out_channels=16, kernel_size=3), #(64,16,72,72)
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2), #(64,16,36,36)
            nn.Dropout(),
    
        )
        
        self.fc1=nn.Linear(16*36*36,256)
        self.fc2=nn.Linear(256,6)
        
    def forward(self,img):
        img=self.conv(img)
        img=img.view(-1,16*36*36)
        img=self.fc1(img)
        img=self.fc2(img)
        return img

In [ ]:
# optimizer and loss function
net=GarbageNet().to(device)
optimizer=optim.Adam(net.parameters(),lr=0.001, weight_decay=0.001)
criterion=nn.CrossEntropyLoss()
print(net)

In [ ]:
# no of parameters in the network
def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params+=param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(GarbageNet())

In [ ]:
# training and validation
epochs=30

def train_valid(net, train_loader, valid_loader, epochs, criterion):
    train_loss=[] # training loss for every epoch
    valid_loss=[] # validation loss for every epoch
    train_accuracy=[]
    valid_accuracy=[]
    sum_train_loss=0.0 # sum of training losses for every epoch
    sum_valid_loss=0.0 # sum of validation losses for every epoch
    sum_train_accuracy=0.0
    sum_valid_accuracy=0.0
    
    for epoch in range(1,epochs+1):
    
        train_epoch_loss=0.0
        train_epoch_accuracy=0.0
        net.train()
        for i,(images, labels) in enumerate(train_loader):
            images, labels =images.to(device), labels.to(device)
            output = net(images)
            loss = criterion(output, labels)
        
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_epoch_loss = train_epoch_loss + ((1/(i+1)) * (loss.item() - train_epoch_loss))
            
            #_,pred_label = torch.max(output.data,1)
            pred_softmax=torch.softmax(output,1)
            _,prediction = torch.max(pred_softmax,1)
            #train_epoch_accuracy+=int(torch.sum(pred_label==labels.data))
            train_epoch_accuracy+=int(torch.sum(prediction==labels.data))
        
        train_epoch_accuracy=train_epoch_accuracy/train_count

        train_loss.append(train_epoch_loss)
        sum_train_loss+=train_epoch_loss
        
        train_accuracy.append(train_epoch_accuracy)
        sum_train_accuracy+=train_epoch_accuracy
    
        valid_epoch_loss=0.0
        valid_epoch_accuracy=0.0
        with torch.no_grad():
            net.eval()
            for i,(images, labels) in enumerate(valid_loader):
                images, labels = images.to(device), labels.to(device)
                output = net(images)
                loss = criterion(output, labels)
                
                valid_epoch_loss = valid_epoch_loss + ((1/(i+1)) * (loss.item() - valid_epoch_loss))
                #_,predlabel = torch.max(output.data,1)
                pred_softmax=torch.softmax(output,1)
                _,prediction = torch.max(pred_softmax,1)
                #valid_epoch_accuracy+=int(torch.sum(predlabel==labels.data))
                valid_epoch_accuracy+=int(torch.sum(prediction==labels.data))
        
        valid_epoch_accuracy=valid_epoch_accuracy/valid_count
                
        valid_loss.append(valid_epoch_loss)
        sum_valid_loss+=valid_epoch_loss
        
        valid_accuracy.append(valid_epoch_accuracy)
        sum_valid_accuracy+=valid_epoch_accuracy
        
        print("Epoch {}/{}\n Train loss : {} \t Valid loss {}\n Train accuracy : {} \t Valid accuracy : {}\n"
             .format(epoch, epochs, train_epoch_loss, valid_epoch_loss,train_epoch_accuracy, valid_epoch_accuracy))
        
    print("Average training loss after {} epochs : {}".format(epochs, sum_train_loss/epochs))
    print("Average validation loss after {} epochs : {}\n".format(epochs, sum_valid_loss/epochs))
    print("Average training accuracy after {} epochs : {}".format(epochs, sum_train_accuracy/epochs))
    print("Average validation accuracy after {} epochs : {}".format(epochs, sum_valid_accuracy/epochs))
    
    return train_loss, valid_loss, train_accuracy, valid_accuracy

In [ ]:
train_losses, valid_losses, train_acc, valid_acc = train_valid(net, train_loader, valid_loader, epochs, criterion)

In [ ]:
# function for loss and accuracy plot
def plot_imshow(xlabel, ylabel, plot1, plot1lab, plot2, plot2lab):
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.plot(plot1, label=plot1lab)
    plt.plot(plot2, label=plot2lab)
    plt.legend(bbox_to_anchor=(1.1,1.0), loc='upper left')
    plt.show

In [ ]:
# plot training loss and validation loss
print("Training loss and Validation loss plot")
plot_imshow('epochs', 'loss',train_losses, "Train loss", valid_losses, "Validation loss")

In [ ]:
# plot training accuracy and validation accuracy
print("Training accuracy and Validation accuracy plot")
plot_imshow('epochs', 'accuracy',train_acc, "Train accuracy", valid_acc, "Validation accuracy")

In [ ]:
# imshow
def imshow(img,text=None,should_save=False):
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [ ]:
# prediction
def eval(net, test_loader):
    with torch.no_grad():
        net.eval()
        count=100
        correct=0
        correct_count=0
        dataiter = iter(test_loader)
        
        print("Testing...")
        for i in range(count):
            img, label = next(dataiter)
            image=img
            output = net(Variable(img).cuda())
            pred_softmax=torch.softmax(output,1)
            _,prediction = torch.max(pred_softmax,1)
            #print(prediction)
            total = label.size(0)

            # check if prediction and actual label are same
            for j in range(output.size(0)):
                if (prediction[j]==label[j]):
                    correct+=1
                
            correct_count+=correct/total
            correct=0
            imshow(torchvision.utils.make_grid(image),'Pred : {}  Label : {}'.format(classes[prediction.item()],classes[label.item()]))   
        
    return correct_count, count, (correct_count/count)*100

In [ ]:
acc=eval(net, test_loader)
print('{} correct predictions out of {}\nAccuracy : {:.2f}'.format(acc[0],acc[1], acc[2]))